In [6]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np

# internal imports
from tgb.utils.utils import *
from tgb.linkproppred.evaluate import Evaluator
# from modules.decoder import LinkPredictorTGNPL
# from modules.emb_module import GraphAttentionEmbedding
# from modules.msg_func import TGNPLMessage
# from modules.msg_agg import MeanAggregator
# from modules.memory_module import TGNPLMemory, StaticMemory
from modules.hyper_edgebank import HyperEdgeBankPredictor
from modules.neighbor_loader import LastNeighborLoaderGraphmixer
from modules.early_stopping import  EarlyStopMonitor
from test_hyper_edgebank import test_edgebank
from tgb.linkproppred.dataset_pyg import PyGLinkPropPredDataset, PyGLinkPropPredDatasetHyper
from graphmixer import *

from scipy.stats import pearsonr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This notebook is using graphmixer as a static memory version of TGNPL to cross-check the correctness of the graphmixer training. The results below are consistent with what we got in test_model_training.ipynb, as desired. 
# Initialize data

In [2]:
# recreate args
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
args = Namespace(dataset='tgbl-hypergraph',
                 lr=1e-4,
                 bs=2000,  # use larger batch size since we're not training
                 k_value=10,
                 num_epoch=100,
                 seed=1,
                 mem_dim=100,
                 time_dim=10,
                 emb_dim=100,
                 tolerance=1e-6,
                 patience=100,
                 num_run=1,
                 wandb=False,
                 bipartite=False,
                 memory_name='static',
                 emb_name='sum',
                 use_inventory=False,
                 debt_penalty=0,
                 consum_rwd=0,
                 gpu=0,
                 num_train_days=-1,
                 num_layers=2,
                 dropout=0.1, 
                 num_neighbors=300, # so that every time neighbor loader loads in all datapoints
                 use_prev_sampling=True, # since we tgbl-hypergraph dataset is before change to sampling method
                 time_gap=300)
device = torch.device("cpu")

In [3]:
# Initialize dataset - copied from run_experiment() in tgnpl.py
with open(os.path.join(PATH_TO_DATASETS, f"{args.dataset.replace('-', '_')}/{args.dataset}_meta.json"), "r") as f:
    metadata = json.load(f)
# set global data variables
num_nodes = len(metadata["id2entity"])  
num_firms = metadata["product_threshold"]
num_products = num_nodes - num_firms              
dataset = PyGLinkPropPredDatasetHyper(name=args.dataset, root="datasets")
metric = dataset.eval_metric
neg_sampler = dataset.negative_sampler
evaluator = Evaluator(name=args.dataset)
data = dataset.get_TemporalData().to(device)
train_loader, val_loader, test_loader = set_up_data(args, data, dataset)

dataset.load_val_ns()  # load validation negative samples

Dataset tgbl-hypergraph url not found, download not supported yet.
file found, skipping download
Dataset directory is  /lfs/turing1/0/zhiyinl/supply-chains/TGB/tgb/datasets/tgbl_hypergraph
loading processed file
Train: N=214674, 273 days; Val: N=44991, 45 days; Test: N=45612, 47 days


# Try fitting model on tiny data
We construct graphmixer such that it mimics the static memory with additional time varying features. We expect its performances to be similar static (e.g. MRR=1 for test of train data, no neighbors). 

In [4]:
# first 100 data points in val, NO neighbors
mini_loader = TemporalDataLoader(data[dataset.val_mask][:100], batch_size=args.bs)
assert len(mini_loader) == 1
batch = next(iter(mini_loader))
pos_src, pos_prod, pos_dst, pos_t, pos_msg = batch.src, batch.prod, batch.dst, batch.t, batch.msg
print(pos_src.shape)

torch.Size([100])


In [ ]:
# try graphmixer with self.mimic_static_debug=True
model, opt = set_up_model(args, data, device, num_firms=num_firms, num_products=num_products, mimic_static_debug=True)
neighbor_loader = LastNeighborLoaderGraphmixer(num_nodes, num_neighbors=args.num_neighbors, time_gap=args.time_gap, edge_feat_dim=1) # size>=num_nodes so time_gap shouldn't be a problem

# fit model on FIXED negative samples from neg_sampler
for epoch in range(1, 1000):
    loss, logits_loss, inv_loss, debt_loss, consump_rwd_loss, update_loss = train(model, 
                       opt, neighbor_loader, data, mini_loader, device, 
                       loss_name='ce-softmax', update_params=True, 
                       neg_sampler=neg_sampler, split_mode="val", 
                       num_firms=num_firms, num_products=num_products, use_prev_sampling=args.use_prev_sampling)
    if epoch % 10 == 0:
        print(epoch, loss)

  0%|                                                                                                                | 0/1 [00:00<?, ?it/s]/lfs/turing1/0/zhiyinl/supply-chains/TGB/examples/linkproppred/general/graphmixer.py:110: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  neg_batch_list = torch.Tensor(neg_batch_list)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.01s/it]


10 2.4636504650115967


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.51s/it]


20 0.9732151627540588


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.02s/it]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.70s/it]


190 0.0018383386777713895


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.39s/it]


200 0.001636747969314456


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.46s/it]


In [8]:
# test MRR on same data
# graphmixer static: similar result before and after reset
test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products, 
     use_prev_sampling=args.use_prev_sampling)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.45s/it]


1.0

In [9]:
# after reset perfect result
neighbor_loader.reset_state()
test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
     use_prev_sampling=args.use_prev_sampling)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.64s/it]


1.0

In [7]:
model, opt = set_up_model(args, data, device, num_firms=num_firms, num_products=num_products, mimic_static_debug=True)
neighbor_loader = LastNeighborLoaderGraphmixer(num_nodes, num_neighbors=args.num_neighbors, time_gap=args.time_gap, edge_feat_dim=1) # size>=num_nodes so time_gap shouldn't be a problem

# fit model on RANDOM negative samples
# should be able to learn but loss will go down a bit more randomly
for epoch in range(1, 1000):
    loss, logits_loss, inv_loss, debt_loss, consump_rwd_loss, update_loss = train(model, 
                       opt, neighbor_loader, data, mini_loader, device, 
                       loss_name='ce-softmax', update_params=True, 
                       ns_samples=6, num_firms=num_firms, num_products=num_products)
    if epoch % 10 == 0:
        print(epoch, loss)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.62s/it]


10 2.5731804370880127


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.77s/it]


20 1.2648053169250488


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.92s/it]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.77s/it]


980 0.025502679869532585


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.29s/it]


990 0.025405975058674812


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.62s/it]


In [8]:
# test MRR on same data
# graphmixer static: similar result before and after reset
mrr1 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products, 
            use_prev_sampling=args.use_prev_sampling)

# same results after we reset
neighbor_loader.reset_state()
mrr2 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

print(f'MRR: before resetting = {mrr1:.4f}; after resetting = {mrr2:.4f}')

  0%|                                                                                                                | 0/1 [00:00<?, ?it/s]/lfs/turing1/0/zhiyinl/supply-chains/TGB/examples/linkproppred/general/graphmixer.py:110: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  neg_batch_list = torch.Tensor(neg_batch_list)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.03s/it]

MRR: before resetting = 0.9500; after resetting = 0.9500


In [9]:
# first 100 data points in val, WITH time-varying neighbors
mini_loader = TemporalDataLoader(data[dataset.val_mask][:100], batch_size=10)
print(len(mini_loader))
batch = next(iter(mini_loader))
pos_src, pos_prod, pos_dst, pos_t, pos_msg = batch.src, batch.prod, batch.dst, batch.t, batch.msg
print(pos_src.shape)

10
torch.Size([10])


In [10]:
# still trying model with static memory
model, opt = set_up_model(args, data, device, num_firms=num_firms, num_products=num_products, mimic_static_debug=True)
neighbor_loader = LastNeighborLoaderGraphmixer(num_nodes, num_neighbors=args.num_neighbors, time_gap=args.time_gap, edge_feat_dim=1) # size>=num_nodes so time_gap shouldn't be a problem

# fit model on FIXED negative samples from neg_sampler, time-varying neighbors
for epoch in range(1, 1000):
    loss, logits_loss, inv_loss, debt_loss, consump_rwd_loss, update_loss = train(model, opt, neighbor_loader, data, mini_loader, device, 
                       loss_name='ce-softmax', update_params=True, 
                       neg_sampler=neg_sampler, split_mode="val", 
                       num_firms=num_firms, num_products=num_products,
                       use_prev_sampling=args.use_prev_sampling)
    if epoch % 10 == 0:
        print(epoch, loss)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:34<00:00,  3.42s/it]


10 0.07274019252508879


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:33<00:00,  3.39s/it]


20 0.01088059025350958


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:35<00:00,  3.59s/it]


...

430 9.383911628901841e-06


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:42<00:00,  4.28s/it]


440 9.022731478580682e-06


 30%|██████████████████████████████▉                                                                        | 3/10 [00:16<00:37,  5.36s/it]


KeyboardInterrupt: 

In [11]:
# test MRR on same data
# graphmixer static: similar result before and after reset
mrr1 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

# after reset perfect resut
neighbor_loader.reset_state()
mrr2 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

print(f'MRR: before resetting = {mrr1:.4f}; after resetting = {mrr2:.4f}')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.44s/it]

MRR: before resetting = 1.0000; after resetting = 1.0000


In [12]:
# still trying model with static memory
model, opt = set_up_model(args, data, device, num_firms=num_firms, num_products=num_products, mimic_static_debug=True)
neighbor_loader = LastNeighborLoaderGraphmixer(num_nodes, num_neighbors=args.num_neighbors, time_gap=args.time_gap, edge_feat_dim=1) # size>=num_nodes so time_gap shouldn't be a problem

# fit model on RANDOM negative samples, time-varying neighbors
# should be able to learn but loss will go down a bit more randomly
for epoch in range(1, 1000):
    loss, logits_loss, inv_loss, debt_loss, consump_rwd_loss, update_loss = train(model, opt, neighbor_loader, data, mini_loader, device, 
                       loss_name='ce-softmax', update_params=True, 
                       ns_samples=6, num_firms=num_firms, num_products=num_products,
                       use_prev_sampling=args.use_prev_sampling)
    if epoch % 10 == 0:
        print(epoch, loss)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:40<00:00,  4.05s/it]


10 0.28071609027683736


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:52<00:00,  5.20s/it]


20 0.17584584411233664


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:50<00:00,  5.03s/it]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:43<00:00,  4.39s/it]


980 0.001458391357846267


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:42<00:00,  4.27s/it]


990 0.004578629249681399


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:42<00:00,  4.28s/it]


In [13]:
# test MRR on same data
# low cross-entropy loss should correspond to high MRR
# if we don't reset neighbor loader, results will be bad
mrr1 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

# after reset, the result is much better
neighbor_loader.reset_state()
mrr2 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

print(f'MRR: before resetting = {mrr1:.4f}; after resetting = {mrr2:.4f}')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:23<00:00,  2.37s/it]

MRR: before resetting = 0.4290; after resetting = 0.9950
